# PyTorch - homework 1


Please run the whole notebook with your code and submit the `.ipynb` file that includes your answers. 

In [1]:
from termcolor import colored

student_number="1003391"
student_name="Tan Chia Yik"

print(colored("Homework by "  + student_name + ', number: ' + student_number,'red'))

Homework by Tan Chia Yik, number: 1003391


 ## Question 1 -- matrix multiplication

Implement the following mathematical operation on both the CPU and GPU (use Google Colab or another cloud service if you don't have a GPU in your computer). Print:

a) which type of GPU card you have and 

b) show the computation time for both CPU and GPU (using PyTorch). 

c) How much % fast is the GPU? 

 The operation to implement is the dot product $C = B * A^T$

 whereby $A$ is a random matrix of size $20,000 \times 1000$ and $B$ is a random matrix of size $2000 \times 1000$. In addition to the required information asked above:
 
 d) please also print the resulting two $C$ matrices (they should be the same btw). 
 



In [2]:
import torch
import numpy as np
import timeit
import os

# implement solution here
# a) printing type of GPU card
print ("Type of GPU card")
!nvidia-smi
print ("\n")

#b) computation type between CPU and GPU
matrix_A = torch.rand(20000, 1000)
matrix_B = torch.rand(2000, 1000)
matrix_A_T = torch.transpose(matrix_A, 0, 1)

#matrixr Computation
def compute_matrix_mm(matrix_B, matrix_A_T):
  return matrix_B.mm(matrix_A_T)

print ("computation time without GPU")
print (timeit.timeit("compute_matrix_mm(matrix_B, matrix_A_T)", "from __main__ import compute_matrix_mm, matrix_B, matrix_A_T" , number = 10))

using_GPU = os.path.exists('/opt/bin/nvidia-smi') 
if using_GPU:
  print ("computation time with GPU")  
  gpu_test_input_one = matrix_B.cuda()
  gpu_test_input_two = matrix_A_T.cuda()
  print (timeit.timeit("compute_matrix_mm(gpu_test_input_one, gpu_test_input_two)", "from __main__ import compute_matrix_mm, gpu_test_input_one, gpu_test_input_two" , number = 10))





Type of GPU card
Fri Jun 25 07:11:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                       

## Question 2 - grad


Find the gradient (partial derivatives) of the function $g(w)$ below. 

Let  $w=[w_1,w_2]^T$

Consider  $g(w)=2w_1w_2+w_2cos(w_1)$

a) In PyTorch, compute:   $\nabla g(w)$ 

 and verify that $\nabla g([\pi,1])=[2,2\pi−1]^T$ using the grad function, whereby the first position is the partial for $w_1$ and the second position is the partial for $w_2$. 

b) You can also write a function to manually calculate these partial derivatives! You can review your differential equations math at [here](https://www.wolframalpha.com/input/?i=derivative+y+cos%28x%29) and implement this is a second function below to verify that it comes to the same solution. 


In [7]:
# write your solution here\
import math

def g(w):
  return (2*w[0]*w[1] + w[1]*math.cos(w[0]))

w = torch.tensor([math.pi, 1], requires_grad = True)
x = g(w)
x.backward()

print (w.grad)
print (torch.tensor([2, 2*math.pi - 1]))

tensor([2.0000, 5.2832])
tensor([2.0000, 5.2832])


## Question 3 - dance hit song prediction

Implement logistic regression in PyTorch for the following dance hit song prediction training dataset: 
https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv

 * Input variables: a number of audio features (most already standardized so don't worry about that)
 * Target variable: Topclass1030: 
   * 1 means it was a top 10 hit song; 
   * 0 means it never went above top 30 position.

This dataset is derived from my paper on dance hit song prediction, for full description of features have a look at https://arxiv.org/abs/1905.08076. 

Print the evolution of the loss every few epochs and train the model until it converges. 
 
 After training the logistic regression model, calculate the prediction accuracy on the test set: 
 https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv








In [33]:
# Your code here
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import io

# load data
all_data = pd.read_csv(io.BytesIO(uploaded['herremans_hit_1030training.csv']))
labels = all_data.iloc[:,-1] #Gets the target values, 1 means top 10 hit song, 0 means not top 10 hit song
labels = torch.Tensor(labels.values).reshape(-1,1)
train_data = all_data.drop('Topclass1030', axis=1) #Removes labels from dataset
train_data = torch.Tensor(train_data.values) 

num_input_features = train_data.size(1) #has an output dimension of 1

# define logistic regression model
class LogisticReg(nn.Module):
  def __init__(self, input_size, output_size = 1):
    super(LogisticReg, self).__init__()
    self.linear = nn.Linear(input_size, output_size)
  
  def forward(self, x):
    out = self.linear(x) #passes through a lienar layer
    out = torch.sigmoid(out) # applys sigmoid function for logistic regression
    return out

logreg_clf = LogisticReg(num_input_features)

epochs = 501
learning_rate = 0.01
loss_function = nn.MSELoss() #Mean Squared Error loss
optimizer = torch.optim.SGD(logreg_clf.parameters(), lr=learning_rate)

for i in range(epochs):
    logreg_clf.train()
    optimizer.zero_grad()
    
    # Forward Pass
    output_y = logreg_clf(train_data)

    # Compute Loss
    loss = loss_function(output_y, labels)

    # Backward pass
    loss.backward()
    optimizer.step()

    if i % 20 == 0:
        print(f"Epoch - {i} , Loss - {loss.item()}")


# train model


Epoch - 0 , Loss - 0.2985572814941406
Epoch - 20 , Loss - 0.26793330907821655
Epoch - 40 , Loss - 0.24964073300361633
Epoch - 60 , Loss - 0.23888833820819855
Epoch - 80 , Loss - 0.23216494917869568
Epoch - 100 , Loss - 0.22756674885749817
Epoch - 120 , Loss - 0.22413977980613708
Epoch - 140 , Loss - 0.22140339016914368
Epoch - 160 , Loss - 0.2191057950258255
Epoch - 180 , Loss - 0.21710748970508575
Epoch - 200 , Loss - 0.21532656252384186
Epoch - 220 , Loss - 0.2137119323015213
Epoch - 240 , Loss - 0.21223002672195435
Epoch - 260 , Loss - 0.21085752546787262
Epoch - 280 , Loss - 0.20957763493061066
Epoch - 300 , Loss - 0.20837777853012085
Epoch - 320 , Loss - 0.2072482407093048
Epoch - 340 , Loss - 0.20618131756782532
Epoch - 360 , Loss - 0.20517072081565857
Epoch - 380 , Loss - 0.20421120524406433
Epoch - 400 , Loss - 0.20329831540584564
Epoch - 420 , Loss - 0.20242832601070404
Epoch - 440 , Loss - 0.20159783959388733
Epoch - 460 , Loss - 0.2008039504289627
Epoch - 480 , Loss - 0.2000

Run the below code to test the accuracy of your model on the training set: 

In [34]:
import pandas as pd
import io

test = pd.read_csv(io.BytesIO(uploaded['herremans_hit_1030training.csv']))
#test = pd.read_csv("data/herremans_hit_1030training.csv")

labels = test.iloc[:,-1]
test = test.drop('Topclass1030', axis=1)
testdata = torch.Tensor(test.values)
testlabels = torch.Tensor(labels.values).view(-1,1)

TP = 0
TN = 0
FN = 0
FP = 0

for i in range(0, testdata.size()[0]): 
  # print(testdata[i].size())
  Xtest = torch.Tensor(testdata[i])
  y_hat = logreg_clf(Xtest)
  
  if y_hat > 0.5:
    prediction = 1
  else: 
    prediction = 0

  if (prediction == testlabels[i]):
    if (prediction == 1):
      TP += 1
    else: 
      TN += 1

  else:
    if (prediction == 1):
      FP += 1
    else: 
      FN += 1

print("True Positives: {0}, True Negatives: {1}".format(TP, TN))
print("False Positives: {0}, False Negatives: {1}".format(FP, FN))
rate = TP/(FN+TP)
print("Class specific accuracy of correctly predicting a hit song is {0}".format(rate))

True Positives: 183, True Negatives: 44
False Positives: 74, False Negatives: 20
Class specific accuracy of correctly predicting a hit song is 0.9014778325123153


In [5]:
#to upload filed to google colab

from google.colab import files
uploaded = files.upload()

Saving herremans_hit_1030training.csv to herremans_hit_1030training.csv
